# https://github.com/kernoelpanic/smartenv

# Smart Contract Crash Course - Basic interaction with `ganache-cli`

The most common client lib to interface with ethereum clients (`geth`,`ganache`,...) is **web3**.
* [web3.js](https://github.com/ethereum/web3.js/)
    - node.js version
    - [API](https://web3js.readthedocs.io/)
* [web3py](https://github.com/ethereum/web3.py)
    - python version
    - as I prefer python, we will use this one ;)
    - [API](https://web3py.readthedocs.io/)

For the tutorial we will only interact with `ganache` which uses his own local development testnet blockchain. 
* [ganache-cli](https://github.com/trufflesuite/ganache-cli/tree/master)
* [API](https://github.com/trufflesuite/ganache-cli#implemented-methods)

## Connect via RPC / HTTP
Import the web3 libarary and connect to a running `ganache-cli` node via `HTTPProvider`
For the tutorial configuration the docker container of ganache is located at `172.18.0.2`.

In [313]:
import web3

w3_ganache = web3.Web3(web3.Web3.HTTPProvider("http://172.18.0.2:8545"))
# check if connection was successful
assert w3_ganache.isConnected()

In [314]:
# Client version, should be TestRPC
w3_ganache.clientVersion

'EthereumJS TestRPC/v2.13.2/ethereum-js'

In [315]:
# display network ID of client you are connected to, should be a random int
w3_ganache.net.version

'1625154330440'

In [316]:
# number of connected peers to our peer should be 0
w3_ganache.net.peerCount

0

In [317]:
w3 = w3_ganache

## Blocks

In [318]:
# return current blockchain head of node
# Since ganache "simulates" blokchain,
# blocks are mined on demand.
# Therefore, no block has been mined yet
w3.eth.blockNumber

33

In [319]:
# block 0 ist the genesis block
genesis_block = w3.eth.getBlock(0)
genesis_block

AttributeDict({'number': 0,
 'hash': HexBytes('0x759b931f04184d5ca2849a8c79b509ad2e1d18152823f2c370d4c4dfbf98c9a1'),
 'parentHash': HexBytes('0x0000000000000000000000000000000000000000000000000000000000000000'),
 'mixHash': HexBytes('0x0000000000000000000000000000000000000000000000000000000000000000'),
 'nonce': HexBytes('0x0000000000000000'),
 'sha3Uncles': HexBytes('0x1dcc4de8dec75d7aab85b567b6ccd41ad312451b948a7413f0a142fd40d49347'),
 'logsBloom': HexBytes('0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000'),
 'transactionsRoo

As you see, the genesis block is special as it is the only block with no parent hash:

In [320]:
genesis_block['number']

0

In [321]:
genesis_block['parentHash']

HexBytes('0x0000000000000000000000000000000000000000000000000000000000000000')

In [322]:
# instruct our test blockchain to mine a block
w3.provider.make_request('evm_mine',params='')

{'id': 6, 'jsonrpc': '2.0', 'result': '0x0'}

In [323]:
w3.eth.blockNumber

34

In [324]:
first_block = w3.eth.getBlock(1)
first_block

AttributeDict({'number': 1,
 'hash': HexBytes('0xec54072b66e6263c7b840555a6cf129e90d74ebb9788ca7fea2ad56c563268fa'),
 'parentHash': HexBytes('0x759b931f04184d5ca2849a8c79b509ad2e1d18152823f2c370d4c4dfbf98c9a1'),
 'mixHash': HexBytes('0x0000000000000000000000000000000000000000000000000000000000000000'),
 'nonce': HexBytes('0x0000000000000000'),
 'sha3Uncles': HexBytes('0x1dcc4de8dec75d7aab85b567b6ccd41ad312451b948a7413f0a142fd40d49347'),
 'logsBloom': HexBytes('0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000'),
 'transactionsRoo

In [325]:
first_block['number']

1

In [326]:
first_block['parentHash']

HexBytes('0x759b931f04184d5ca2849a8c79b509ad2e1d18152823f2c370d4c4dfbf98c9a1')

In [327]:
# our node is configured to mine blocks 
w3.eth.mining

True

In [328]:
# but since it is a fake chain we have no hashrate
w3.eth.hashrate

0

## Externally Owned Accounts

Accounts are your **external accounts** consisting of public and private keys. 
The ethereum address are the last 40 hex characters (20 bytes) of the hash (Keccak-256) of the public key prefixed with `0x`

`ganache` generates a list of accounts on startup. 
For the challenges you will recieve your personal account keypair which you have to copy in your `geth` datadir folder to import it. 

In [329]:
w3.eth.accounts

['0x90F8bf6A479f320ead074411a4B0e7944Ea8c9C1',
 '0xFFcf8FDEE72ac11b5c542428B35EEF5769C409f0',
 '0x22d491Bde2303f2f43325b2108D26f1eAbA1e32b',
 '0xE11BA2b4D45Eaed5996Cd0823791E0C93114882d',
 '0xd03ea8624C8C5987235048901fB614fDcA89b117',
 '0x95cED938F7991cd0dFcb48F0a06a40FA1aF46EBC',
 '0x3E5e9111Ae8eB78Fe1CC3bb8915d5D461F3Ef9A9',
 '0x28a8746e75304c0780E011BEd21C72cD78cd535E',
 '0xACa94ef8bD5ffEE41947b4585a84BdA5a3d3DA6E',
 '0x1dF62f291b2E969fB0849d99D9Ce41e2F137006e']

In [330]:
len(w3.eth.accounts)

10

In [331]:
print("address length with 0x at the beginning = ",len(w3.eth.accounts[0]))
w3.eth.accounts[0]

address length with 0x at the beginning =  42


'0x90F8bf6A479f320ead074411a4B0e7944Ea8c9C1'

**Note:** Ethereum uses Keccak-256 but often refers to it as SHA3, but the output of SHA3 as specified by NIST is different!

In [332]:
from sha3 import sha3_256
sha3_256(b'').hexdigest()

'a7ffc6f8bf1ed76651c14756a061d662f580ff4de43b49fa82d80a4b80f8434a'

In [333]:
from sha3 import keccak_256
keccak_256(b'').hexdigest()

'c5d2460186f7233c927e7db2dcc703c0e500b653ca82273b7bfad8045d85a470'

In [334]:
w3.sha3(text="").hex()[2:]

'c5d2460186f7233c927e7db2dcc703c0e500b653ca82273b7bfad8045d85a470'

In [335]:
assert keccak_256(b'').hexdigest() == w3.sha3(text="").hex()[2:]

The default address (also used to receive rewards from mining blocks) is usually the first in the list of accounts and usually also accessible via `w3.eth.coinbase`

In [336]:
assert w3.eth.accounts[0] == w3.eth.coinbase
w3.eth.coinbase

'0x90F8bf6A479f320ead074411a4B0e7944Ea8c9C1'

It is good practise to specifically assing a `default_account` some functions might cause problems if this is not explicitly assigned.

In [337]:
w3.eth.default_account = w3.eth.accounts[0]
w3.eth.default_account 

'0x90F8bf6A479f320ead074411a4B0e7944Ea8c9C1'

There is also a helper method to check if a string is an ethereum address.

In [338]:
w3.isAddress(w3.eth.accounts[0])

True

In [339]:
test_addr = '0xd3cda913deb6f67967b99d67acdfa1712c293601'
print("address\t\t\t: {}\nlength\t\t\t: {}\nw3.isAddress()\t\t: {}".format(test_addr,
                                              len(test_addr),
                                              w3.isAddress(test_addr)))
print()
print("\nWrong address:")
print("address\t\t\t: {}\nlength\t\t\t: {}\nw3.isAddress()\t\t: {}".format(test_addr[:-1],
                                              len(test_addr[:-1]),
                                              w3.isAddress(test_addr[:-1])))

address			: 0xd3cda913deb6f67967b99d67acdfa1712c293601
length			: 42
w3.isAddress()		: True


Wrong address:
address			: 0xd3cda913deb6f67967b99d67acdfa1712c29360
length			: 41
w3.isAddress()		: False


**Note:** The capital letters of an address encode an optional checksum
https://github.com/ethereum/EIPs/blob/master/EIPS/eip-55.md

In [340]:
w3.isChecksumAddress(test_addr)
print("address\t\t\t: {}\nlength\t\t\t: {}\nw3.isAddress()\t\t: {}\nw3.isChecksumAddress()\t: {}".format(test_addr,
                                              len(test_addr),
                                              w3.isAddress(test_addr),
                                              w3.isChecksumAddress(test_addr)))

address			: 0xd3cda913deb6f67967b99d67acdfa1712c293601
length			: 42
w3.isAddress()		: True
w3.isChecksumAddress()	: False


In [341]:
test_addr = w3.toChecksumAddress(test_addr)
print("address\t\t\t: {}\nlength\t\t\t: {}\nw3.isAddress()\t\t: {}\nw3.isChecksumAddress()\t: {}".format(test_addr,
                                              len(test_addr),
                                              w3.isAddress(test_addr),
                                              w3.isChecksumAddress(test_addr)))

address			: 0xd3CdA913deB6f67967B99D67aCDFa1712C293601
length			: 42
w3.isAddress()		: True
w3.isChecksumAddress()	: True


### Example: Create externally owned account 

To see what an externally controlled account actually is, lets create one. 
First useing `geth` then with `web3.py`.

In [342]:
!pwd 

/smartenv/examples/Greeter


In [343]:
TEST_PATH = "/smartenv/gitignore/testaccount/"

In [344]:
!echo {TEST_PATH} # or !echo $TEST_PATH

/smartenv/gitignore/testaccount/


In [345]:
!mkdir -p {TEST_PATH}

In [346]:
!echo "password" > {TEST_PATH}pwd.txt

In [347]:
!geth --maxpeers 0 --datadir={TEST_PATH} --password={TEST_PATH}/pwd.txt account new

INFO [07-06|09:12:06.017] Maximum peer count                       ETH=0 LES=0 total=0
INFO [07-06|09:12:06.017] Smartcard socket not found, disabling    err="stat /run/pcscd/pcscd.comm: no such file or directory"

Your new key was generated

Public address of the key:   0x5e9a5C8a7836C73401730E1FF968B1Ee71Ea9452
Path of the secret key file: /smartenv/gitignore/testaccount/keystore/UTC--2021-07-06T07-12-06.018324089Z--5e9a5c8a7836c73401730e1ff968b1ee71ea9452

- You can share your public address with anyone. Others need it to interact with you.
- You must NEVER share the secret key with anyone! The key controls access to your funds!
- You must BACKUP your key file! Without the key, it's impossible to access account funds!
- You must REMEMBER your password! Without the password, it's impossible to decrypt the key!



In [348]:
keystore_file = !ls {TEST_PATH}/keystore
keystore_file[0]

'UTC--2021-06-12T18-08-28.982657680Z--84243c9b687af1c51e2b20f92c8c8de6efde05cf'

In [349]:
with open(TEST_PATH + 'keystore/' + keystore_file[0]) as keyfile:
    keyfile_json = keyfile.read()

keyfile_json

'{"address":"84243c9b687af1c51e2b20f92c8c8de6efde05cf","crypto":{"cipher":"aes-128-ctr","ciphertext":"62aed34bba8a32334ac592cccbf62dd9a042141140df8f7786347be6fc0aa9a6","cipherparams":{"iv":"8f8c66531af03f2391e207d4d9b36266"},"kdf":"scrypt","kdfparams":{"dklen":32,"n":262144,"p":1,"r":8,"salt":"8ee99e2c9839d11ea44d40a2b3735a0992abf310f613015188a7b9bf63a1e296"},"mac":"bfc374f72d4d27d678b2d7cf3a0c5cb689fc911f0a026a91e7765a8fe874a4e8"},"id":"c91de86c-44de-4e0a-8867-3f48054bd5b0","version":3}'

In [350]:
private_key = w3.eth.account.decrypt(keyfile_json,"password")

In [351]:
private_key.hex()

'0xd23750555baac94915154e1bbeb0cc7166c80d9e7966a9484acfe8dea2e6ad60'

In [352]:
account = w3.eth.account.privateKeyToAccount(private_key)

In [353]:
account.address

'0x84243c9B687af1c51E2B20f92c8C8dE6EFdE05Cf'

In [354]:
account.privateKey.hex()

'0xd23750555baac94915154e1bbeb0cc7166c80d9e7966a9484acfe8dea2e6ad60'

In [355]:
account.encrypt("password")

{'address': '84243c9b687af1c51e2b20f92c8c8de6efde05cf',
 'crypto': {'cipher': 'aes-128-ctr',
  'cipherparams': {'iv': '4de11e264318a79aeb73b06b1dd9f217'},
  'ciphertext': '5d1fcca969bdd4444e1e01ce26588e8222538d14dbf3eeb92feae01a64f20506',
  'kdf': 'scrypt',
  'kdfparams': {'dklen': 32,
   'n': 262144,
   'r': 1,
   'p': 8,
   'salt': '805afc3c514e30b4fb7104c4799d9a8b'},
  'mac': '35788445a821967944f32e37449dba46a17558b57b1a94263dcdb897beb020b2'},
 'id': 'e0eadefd-e81c-4209-b67a-107663143741',
 'version': 3}

Lets create an account directly with web3.py:

In [356]:
account = w3.eth.account.create("random string to increase entropy")

In [357]:
account.address

'0xEd049867094891fE450d2146c2ab8eA239f3cB58'

In [358]:
account.privateKey

HexBytes('0x310953518eda7ebf9cfaa03fe19098b22ae9faf13a9bd396bd5a07fdab54e034')

In [359]:
account.encrypt("password")

{'address': 'ed049867094891fe450d2146c2ab8ea239f3cb58',
 'crypto': {'cipher': 'aes-128-ctr',
  'cipherparams': {'iv': '403a6689f680cbdc05c5ee9698416f73'},
  'ciphertext': 'a637d38252322dddd185ba8ac662335cb4d5fb1d86a38809ad782cb96e6b1e90',
  'kdf': 'scrypt',
  'kdfparams': {'dklen': 32,
   'n': 262144,
   'r': 1,
   'p': 8,
   'salt': '5b8caccfa76b000ab652f0723c6831f1'},
  'mac': 'd176e1d6257f8745f56f3d5f78acbad51b70d8381d2906332fbfac05d5d34a45'},
 'id': '8e02d309-e47f-4a3b-b2e2-d8e4425bdc7c',
 'version': 3}

## Balance
To query the balance of an account or contract the function `eth.getBalance()` can be used with the respective address.

**Note:** Per default all balances are in *Wei* i.e., the smallest unit of value in Ethereum. 
$$
    1 \textrm{ ether} = 10^{18} \textrm{ wei} 
$$

In [360]:
balance = w3.eth.getBalance(w3.eth.accounts[0])
balance

94896083339999999999

In [361]:
balance * 10**-18

94.89608334

In [362]:
# Directly output the balance in ether from wei
w3.fromWei(balance,'ether')

Decimal('94.896083339999999999')

In [363]:
# Directly output the balance in wei from ether
w3.toWei(100,'ether')

100000000000000000000

In [364]:
def getBalance(address):
    return w3.fromWei(w3.eth.getBalance(address),'ether')

In [365]:
getBalance(w3.eth.accounts[0]) 

Decimal('94.896083339999999999')

In [366]:
# each of the ganache test account has initially 100 ether balance
for account in w3.eth.accounts:
    print(getBalance(account))

94.896083339999999999
103
100
100
100
100
100
100
100
100


## Payment Transaction and Gas

We now send a transaction that just transfers value i.e., ether from one account to another. Thereby, `ganache` automatically mines a block once it sees that a transactions requires confirmation. 

**Note:** When connected to `geth` or `openethereum` you have to unlock an account first to send a transaction since the private keys are stored encrypted per default and protected with a password. In `ganache` this is not necessary since it is a development environment.

In [367]:
# required in geth but not in ganache
# Parameters are:
# * account
# * password
# * duration of unlock (if 0 then forever)
w3.geth.personal.unlockAccount(w3.eth.accounts[0],"",0)

True

In [368]:
w3.eth.blockNumber

34

In [369]:
w3.eth.sendTransaction({'from':w3.eth.accounts[0], 
                        'to':w3.eth.accounts[1], 
                        'value':10**18})

HexBytes('0x11b326ae6eddf33026e60afabb5447dec0203fb0965aef7ef8584ff48c8f09f2')

In [370]:
blkNumber = w3.eth.blockNumber
blkNumber

35

In [371]:
# the balance is reduced by more than 1 ether
# Why do you think this is the case?
getBalance(w3.eth.accounts[0])

Decimal('93.895663339999999999')

In [372]:
getBalance(w3.eth.accounts[1])

Decimal('104')

In [373]:
# get first (and only) transaction in the previously mined block
tx = w3.eth.getTransactionByBlock(blkNumber,0)
tx 

AttributeDict({'hash': HexBytes('0x11b326ae6eddf33026e60afabb5447dec0203fb0965aef7ef8584ff48c8f09f2'),
 'nonce': 30,
 'blockHash': HexBytes('0x014072a658f5ead8f2669e8b669a3aa163efedf33a157a3de3dcf81fb71e7e7b'),
 'blockNumber': 35,
 'transactionIndex': 0,
 'from': '0x90F8bf6A479f320ead074411a4B0e7944Ea8c9C1',
 'to': '0xFFcf8FDEE72ac11b5c542428B35EEF5769C409f0',
 'value': 1000000000000000000,
 'gas': 121000,
 'gasPrice': 20000000000,
 'input': '0x',
 'v': 38,
 'r': HexBytes('0x5ec102318e7577e2ae9b9faf33fa93235ccb54df8865cddde815b54124eb1e56'),
 's': HexBytes('0x0b082b75a511d4fc92a7df14fe9e9f40e3e6be799fbcafc9d2cd0bfe0b14c5c4')})

In [374]:
# get block and see the transaction id and gas used below
blk = w3.eth.getBlock(blkNumber)
blk

AttributeDict({'number': 35,
 'hash': HexBytes('0x014072a658f5ead8f2669e8b669a3aa163efedf33a157a3de3dcf81fb71e7e7b'),
 'parentHash': HexBytes('0xf2b7ea1505da95f5adf4c869510c9f6394c3231a8b73101f1547ad1375dc6a67'),
 'mixHash': HexBytes('0x0000000000000000000000000000000000000000000000000000000000000000'),
 'nonce': HexBytes('0x0000000000000000'),
 'sha3Uncles': HexBytes('0x1dcc4de8dec75d7aab85b567b6ccd41ad312451b948a7413f0a142fd40d49347'),
 'logsBloom': HexBytes('0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000'),
 'transactionsRo

In [375]:
# get the transaction hash (i.e., its ID) of the first (and only) transaction in the block
tx_hash_from_block = blk["transactions"][0].hex()
tx_hash_from_block

'0x11b326ae6eddf33026e60afabb5447dec0203fb0965aef7ef8584ff48c8f09f2'

In [376]:
tx_hash_from_tx = tx["hash"].hex() # hash of the transaction
tx_hash_from_tx

'0x11b326ae6eddf33026e60afabb5447dec0203fb0965aef7ef8584ff48c8f09f2'

In [377]:
assert tx_hash_from_tx == tx_hash_from_block
tx_hash = tx_hash_from_tx

In [378]:
# Query the transaction by its hash is also possible:
tx = w3.eth.getTransaction(tx_hash)
tx

AttributeDict({'hash': HexBytes('0x11b326ae6eddf33026e60afabb5447dec0203fb0965aef7ef8584ff48c8f09f2'),
 'nonce': 30,
 'blockHash': HexBytes('0x014072a658f5ead8f2669e8b669a3aa163efedf33a157a3de3dcf81fb71e7e7b'),
 'blockNumber': 35,
 'transactionIndex': 0,
 'from': '0x90F8bf6A479f320ead074411a4B0e7944Ea8c9C1',
 'to': '0xFFcf8FDEE72ac11b5c542428B35EEF5769C409f0',
 'value': 1000000000000000000,
 'gas': 121000,
 'gasPrice': 20000000000,
 'input': '0x',
 'v': 38,
 'r': HexBytes('0x5ec102318e7577e2ae9b9faf33fa93235ccb54df8865cddde815b54124eb1e56'),
 's': HexBytes('0x0b082b75a511d4fc92a7df14fe9e9f40e3e6be799fbcafc9d2cd0bfe0b14c5c4')})

In [379]:
# Get the transaction receipt for the respective transaction hash:
tx_receipt = w3.eth.get_transaction_receipt(tx_hash)
tx_receipt

AttributeDict({'transactionHash': HexBytes('0x11b326ae6eddf33026e60afabb5447dec0203fb0965aef7ef8584ff48c8f09f2'),
 'transactionIndex': 0,
 'blockHash': HexBytes('0x014072a658f5ead8f2669e8b669a3aa163efedf33a157a3de3dcf81fb71e7e7b'),
 'blockNumber': 35,
 'from': '0x90F8bf6A479f320ead074411a4B0e7944Ea8c9C1',
 'to': '0xFFcf8FDEE72ac11b5c542428B35EEF5769C409f0',
 'gasUsed': 21000,
 'cumulativeGasUsed': 21000,
 'contractAddress': None,
 'logs': [],
 'status': 1,
 'logsBloom': HexBytes('0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000'

* `gasUsed` is the gas used by this transaction
* `cumulativeGasUsed` 
> is the sum of gasUsed by this transaction and all preceding transactions in the same block.

In [380]:
gasUsed_from_block = blk["gasUsed"] # gas used within the whole block
gasUsed_from_block

21000

In [381]:
gasUsed_from_receipt = tx_receipt["gasUsed"]
gasUsed_from_receipt

21000

In [382]:
assert gasUsed_from_block == gasUsed_from_receipt

To estimate the gas costs of a tx, without actually executing it you can use:

In [383]:
# estimate the gas price of an transaction without executing it
w3.eth.estimateGas({'from':w3.eth.accounts[0], 
                    'to':w3.eth.accounts[1], 
                    'value':10**18})

21000

Lets check if the correct amount of gas and thus in the end cryptocurrency units has been used.

In [384]:
# The actual gas price of the tx, 
# i.e., the conversion rate between ether and gas used by the tx
tx_gasPrice = tx["gasPrice"] 
tx_gasPrice

20000000000

In [385]:
# Defined conversion rate between gas and ether specified per transaction
# i.e., default value for how much wei a transaction is willing to pay for as configured in the client 
assert tx_gasPrice == w3.eth.gasPrice 

In [386]:
# gasUsed(by the transaction) * gasPrice(specified in tx)
# gives the wei it costs to send it 
# times 10^18 gives the ether
tx_costs = (21000 * 20000000000) * 10**-18
tx_costs

0.00042

This is exaclty the missing cost on the sender side: 

In [387]:
float(getBalance(w3.eth.accounts[0]))

93.89566334

In [388]:
float(getBalance(w3.eth.accounts[0])) + tx_costs

93.89608334

In [389]:
# Helper function to quickly send some ether to an address:
def sendEther(_value,_to):
    tx_hash = w3.eth.sendTransaction({'from':w3.eth.accounts[0], 
                                      'to':w3.toChecksumAddress(_to), 
                                      'value':w3.toWei(_value,"ether")})
    return tx_hash

# Contract deployment and interaction (the hard way)

Now its time to deploy our first contract.
For the first time we do it the hard way.

## Solidity and its compiler (solc)
Solidity is the high level language:
* https://docs.soliditylang.org/
* https://solidity-by-example.org/

In [390]:
!solc --version

solc, the solidity compiler commandline interface
Version: 0.8.4+commit.c7e474f2.Linux.g++


In [391]:
!solc --help

solc, the Solidity commandline compiler.

This program comes with ABSOLUTELY NO WARRANTY. This is free software, and you
are welcome to redistribute it under certain conditions. See 'solc --license'
for details.

Usage: solc [options] [input_file...]
Compiles the given Solidity input files (or the standard input if none given or
"-" is used as a file name) and outputs the components specified in the options
at standard output or in files in the output directory, if specified.
Imports are automatically read from the filesystem, but it is also possible to
remap paths using the context:prefix=path syntax.
Example:
solc --bin -o /tmp/solcoutput dapp-bin=/usr/local/lib/dapp-bin contract.sol

General Information:
  --help               Show help message and exit.
  --version            Show version and exit.
  --license            Show licensing information and exit.

Input Options:
  --base-path path     Use the given path as the root of the source tree 
                       instead of th

In [392]:
EXAMPLE_PATH="/smartenv/examples/"

In [393]:
cd {EXAMPLE_PATH}/Greeter 

/smartenv/examples/Greeter


* Version Pragma: 
    + https://docs.soliditylang.org/en/v0.8.5/layout-of-source-files.html?highlight=pragma#version-pragma
* SPX-License-Identifier: 
    + https://docs.soliditylang.org/en/v0.8.5/layout-of-source-files.html?highlight=SPX-License#spdx-license-identifier
    + https://spdx.dev/

In [394]:
# execute console command and print Greeter.sol
!cat {EXAMPLE_PATH}/Greeter/Greeter.sol

// SPDX-License-Identifier: MIT
pragma solidity ^0.8.4;

contract Greeter {
    bool private greetbool;
    string public greeting;
    uint256 public greetbit;

    constructor() {
        greeting = 'Hello';
    }

    function setGreeting(string memory _greeting) public {
        greeting = _greeting;
    }

    function getGreeting() view public returns (string memory) {
        return greeting;
    }

    function setGreetbool(bool _bool) public {
        greetbool = _bool;
    }

    function getGreetbool() view public returns (bool) {
        return greetbool; 
    }

    function setGreetbit(uint256 _bit) public {
        greetbit = _bit;
    }

    fallback() external payable {
        greetbit = greetbit ^ 1;
    }

    receive() external payable {
        greetbool = !greetbool;
    }
}


In [395]:
!solc --bin --overwrite -o ./ Greeter.sol

Compiler run successful. Artifact(s) can be found in directory ./.


In [396]:
!cat Greeter.bin

608060405234801561001057600080fd5b506040518060400160405280600581526020017f48656c6c6f0000000000000000000000000000000000000000000000000000008152506001908051906020019061005c929190610062565b50610166565b82805461006e90610105565b90600052602060002090601f01602090048101928261009057600085556100d7565b82601f106100a957805160ff19168380011785556100d7565b828001600101855582156100d7579182015b828111156100d65782518255916020019190600101906100bb565b5b5090506100e491906100e8565b5090565b5b808211156101015760008160009055506001016100e9565b5090565b6000600282049050600182168061011d57607f821691505b6020821081141561013157610130610137565b5b50919050565b7f4e487b7100000000000000000000000000000000000000000000000000000000600052602260045260246000fd5b6107ca806101756000396000f3fe6080604052600436106100745760003560e01c8063501a131d1161004e578063501a131d1461012d578063a413686214610158578063ef690cc014610181578063fe50cc72146101ac576100a2565b80631c55ad30146100b05780633b4f0817146100db5780634f11dbea14610104576100a2565b366100a2576000805490

In [397]:
# assign cat output to python variable
out = !cat Greeter.bin
cbin = out[0]

In [398]:
!solc --abi --overwrite -o ./ Greeter.sol

Compiler run successful. Artifact(s) can be found in directory ./.


In [399]:
!cat Greeter.abi

[{"inputs":[],"stateMutability":"nonpayable","type":"constructor"},{"stateMutability":"payable","type":"fallback"},{"inputs":[],"name":"getGreetbool","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"getGreeting","outputs":[{"internalType":"string","name":"","type":"string"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"greetbit","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"greeting","outputs":[{"internalType":"string","name":"","type":"string"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"uint256","name":"_bit","type":"uint256"}],"name":"setGreetbit","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"bool","name":"_bool","type":"bool"}],"name":"setGreetbool","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"interna

In [400]:
out = !cat Greeter.abi
cabi = out[0]

## Deploy contract

One way to send a deploy transaction is using the web3py libary helper function cunstructor.
This requires:
* The ABI of the contract
* The bytecode of the contract

In [401]:
# send transaction to deploy contract with helper function
contract=w3.eth.contract(abi=cabi,
                         bytecode=cbin)
tx_hash = contract.constructor().transact({"from":w3.eth.accounts[0],})
tx_hash

HexBytes('0xec792a1b8fa1b05cc326d9ad6dce366aa600ed91b773ce67cdd135c4128c2174')

Another way which can also estimate the gas costs of the deploy transaction is sending it 
directly via `sendTransaction` and add the previously compiled contract as `data`. This requires:
* The bytecode of the contract

In [402]:
# estimate gas of deployment transaction
w3.eth.estimateGas({"from":w3.eth.accounts[0],"data":cbin,"value":0})

508710

In [403]:
# send transaction and get transaction hash
tx_hash = w3.eth.sendTransaction({"from":w3.eth.accounts[0],"data":cbin,"value":0})
tx_hash

HexBytes('0x41b7862ddcc3d7d0154a0df2f062e29f65302577fe8c70128a0f118284a05a86')

In [404]:
# here you see the compiled contract as data
tx_info = w3.eth.getTransaction(tx_hash)
tx_info

AttributeDict({'hash': HexBytes('0x41b7862ddcc3d7d0154a0df2f062e29f65302577fe8c70128a0f118284a05a86'),
 'nonce': 32,
 'blockHash': HexBytes('0xf0961408ece706a1d14a38f8c0595bce0eab175648998dfa0f43573b75ee92c7'),
 'blockNumber': 37,
 'transactionIndex': 0,
 'from': '0x90F8bf6A479f320ead074411a4B0e7944Ea8c9C1',
 'to': None,
 'value': 0,
 'gas': 608710,
 'gasPrice': 20000000000,
 'input': '0x608060405234801561001057600080fd5b506040518060400160405280600581526020017f48656c6c6f0000000000000000000000000000000000000000000000000000008152506001908051906020019061005c929190610062565b50610166565b82805461006e90610105565b90600052602060002090601f01602090048101928261009057600085556100d7565b82601f106100a957805160ff19168380011785556100d7565b828001600101855582156100d7579182015b828111156100d65782518255916020019190600101906100bb565b5b5090506100e491906100e8565b5090565b5b808211156101015760008160009055506001016100e9565b5090565b6000600282049050600182168061011d57607f821691505b6020821081141561013157610130610137565

Normally one has to wait till the transaction goes through, with ganache this is almost instant 

In [405]:
# wait till tx goes through 
tx_receipt = w3.eth.waitForTransactionReceipt(tx_hash)
tx_receipt

# other way without wait only successful if mined:
# w3.eth.getTransactionReceipt(tx_hash)

AttributeDict({'transactionHash': HexBytes('0x41b7862ddcc3d7d0154a0df2f062e29f65302577fe8c70128a0f118284a05a86'),
 'transactionIndex': 0,
 'blockHash': HexBytes('0xf0961408ece706a1d14a38f8c0595bce0eab175648998dfa0f43573b75ee92c7'),
 'blockNumber': 37,
 'from': '0x90F8bf6A479f320ead074411a4B0e7944Ea8c9C1',
 'to': None,
 'gasUsed': 508710,
 'cumulativeGasUsed': 508710,
 'contractAddress': '0x2D8BE6BF0baA74e0A907016679CaE9190e80dD0A',
 'logs': [],
 'status': 1,
 'logsBloom': HexBytes('0x0000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000

In [406]:
# the important thing from the tx receipt is the address at which 
# the contract was deployed
caddress = tx_receipt["contractAddress"]
caddress

'0x2D8BE6BF0baA74e0A907016679CaE9190e80dD0A'

In [407]:
# double check if the gasUsed was really our estimate
blk = w3.eth.getBlock(tx_receipt["blockNumber"])
blk["gasUsed"]

508710

## Interact with deployed contract

To get an instance of a deployed contract at its current state we require:
* The contract address `caddress`
* Some ABI information `cabi` either from compiling the contract like done previously or reconstructed through reverse engineering of the bytecode stored in the blockchain.  

In [408]:
# instanciate a deployed contract
instance = w3.eth.contract(address=caddress,
                           abi=cabi)

In [409]:
instance.address # The address of the deployed contract

'0x2D8BE6BF0baA74e0A907016679CaE9190e80dD0A'

In [410]:
instance.abi # The ABI of the deplyed contract

[{'inputs': [], 'stateMutability': 'nonpayable', 'type': 'constructor'},
 {'stateMutability': 'payable', 'type': 'fallback'},
 {'inputs': [],
  'name': 'getGreetbool',
  'outputs': [{'internalType': 'bool', 'name': '', 'type': 'bool'}],
  'stateMutability': 'view',
  'type': 'function'},
 {'inputs': [],
  'name': 'getGreeting',
  'outputs': [{'internalType': 'string', 'name': '', 'type': 'string'}],
  'stateMutability': 'view',
  'type': 'function'},
 {'inputs': [],
  'name': 'greetbit',
  'outputs': [{'internalType': 'uint256', 'name': '', 'type': 'uint256'}],
  'stateMutability': 'view',
  'type': 'function'},
 {'inputs': [],
  'name': 'greeting',
  'outputs': [{'internalType': 'string', 'name': '', 'type': 'string'}],
  'stateMutability': 'view',
  'type': 'function'},
 {'inputs': [{'internalType': 'uint256', 'name': '_bit', 'type': 'uint256'}],
  'name': 'setGreetbit',
  'outputs': [],
  'stateMutability': 'nonpayable',
  'type': 'function'},
 {'inputs': [{'internalType': 'bool', '

### Read storage from deployed contract

In [411]:
# view functions dont change state (just read it) and can therefore be executed to read state
# without performing a transaction on the blockchain
print(instance.functions.getGreeting().call())
print(instance.functions.getGreetbool().call())

Hello
False


In [412]:
# the value of public variables can also be read directly without a getter function
print(instance.functions.greeting().call())
print(instance.functions.greetbit().call())

Hello
0


In [413]:
# the value of private variables cannot be queried directly,
# but this does not offer protection against reads 
import sys
try:
    print(instance.functions.greetbool().call())
except:
    e = sys.exc_info()[0]
    print(e)

<class 'web3.exceptions.ABIFunctionNotFound'>


To read the first value from the storage (in our case the bool variable) we can use
the following function (even if there would be not getter function for it):

In [414]:
assert instance.functions.getGreetbool().call() == False
print(instance.functions.getGreetbool().call())

False


In [415]:
assert w3.eth.getStorageAt(caddress,0).hex() == "0x00"
w3.eth.getStorageAt(caddress,0)

HexBytes('0x00')

No we execute a transaction and see if the bool value has changed:

In [416]:
tx_hash = instance.functions.setGreetbool(True).transact({"from":w3.eth.accounts[0]})
print("Tx hash: ",tx_hash.hex())
print("Now lets wait till tx gets mined (in ganache this is instant) ... ")
tx_receipt = w3.eth.waitForTransactionReceipt(tx_hash)

Tx hash:  0x0944631f760a2d53557e88e89ef10e103c9902d179ac62ebb377ab59b128ee75
Now lets wait till tx gets mined (in ganache this is instant) ... 


In [417]:
assert instance.functions.getGreetbool().call() == True
print(instance.functions.getGreetbool().call())

True


In [418]:
assert w3.eth.getStorageAt(caddress,0).hex() == "0x01"
w3.eth.getStorageAt(caddress,0)

HexBytes('0x01')

### Function input (calldata)

Now lets set the string value:

In [419]:
# Read the current greeting string value
print(instance.functions.greeting().call())

Hello


In [420]:
tx_hash = instance.functions.setGreeting("Hello World!").transact(
    {"from":w3.eth.accounts[0]})
tx_hash.hex()

'0x876dd334818259964a181ebbf65ce8c4ee8b9be7b4bc06e4048307f8e9949853'

In [421]:
# lets check if the output is now different i.e., the value has changed 
print(instance.functions.greeting().call())

Hello World!


In [422]:
tx_info = w3.eth.getTransaction(tx_hash)
tx_info

AttributeDict({'hash': HexBytes('0x876dd334818259964a181ebbf65ce8c4ee8b9be7b4bc06e4048307f8e9949853'),
 'nonce': 34,
 'blockHash': HexBytes('0x5c8542201bf3408ba50ebed26462aa36f36ab9957ec136fd57475eec99da7285'),
 'blockNumber': 39,
 'transactionIndex': 0,
 'from': '0x90F8bf6A479f320ead074411a4B0e7944Ea8c9C1',
 'to': '0x2D8BE6BF0baA74e0A907016679CaE9190e80dD0A',
 'value': 0,
 'gas': 131084,
 'gasPrice': 20000000000,
 'input': '0xa41368620000000000000000000000000000000000000000000000000000000000000020000000000000000000000000000000000000000000000000000000000000000c48656c6c6f20576f726c64210000000000000000000000000000000000000000',
 'v': 38,
 'r': HexBytes('0x86c4ce1f3719de99801b4661f1f7c88073b1bed8b5b750c41afc9038e568c23c'),
 's': HexBytes('0x65a6338f1251a2889e945fcf084211b85aa85ce62c1ed09da84d22c2f3dc5f27')})

In [423]:
input_data = tx_info["input"][2:]
input_data

'a41368620000000000000000000000000000000000000000000000000000000000000020000000000000000000000000000000000000000000000000000000000000000c48656c6c6f20576f726c64210000000000000000000000000000000000000000'

In [424]:
w3.toBytes(hexstr=input_data)

b'\xa4\x13hb\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00 \x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x0cHello World!\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'

Lets decode the calldata input, in this case of one dynamic type parameter string:
* https://solidity.readthedocs.io/en/latest/abi-spec.html#abi

The first 4 bytes (8 hex characters) are the **function signature**. 
See next section for more details. 

The remaining bytes are the string parameter, seperated and aligned in three 32 bytes (256 bit) chunks. 

In [425]:
(len(input_data)-8)//64 

3

In [426]:
[ input_data[8:][i:i+64] for i in range(0,len(input_data),64) ]

['0000000000000000000000000000000000000000000000000000000000000020',
 '000000000000000000000000000000000000000000000000000000000000000c',
 '48656c6c6f20576f726c64210000000000000000000000000000000000000000',
 '']

The first parameter is the offset in bytes to the data part of the first (and only) dynamic parameter string in our case. 
This offset is computed starting from the arguments block. 

In [427]:
offset = 0x20
offset

32

After the offset the length of the string is encoded. 

In [428]:
string_length = 0x0c
string_length

12

The rest is the actual string

In [429]:
# lets decode the input data of the transaction
input_data = bytes.fromhex("48656c6c6f20576f726c6421").decode("utf-8")
print(input_data) # input data string
print("length = ",len(input_data))

Hello World!
length =  12


There is also helper function that can decode and encode function parameters in a more human readable form:

In [430]:
# decode
instance.decode_function_input(tx_info.input)

(<Function setGreeting(string)>, {'_greeting': 'Hello World!'})

In [431]:
# encode
payload= instance.encodeABI(fn_name='setGreeting', args=["Hello World!"])
payload

'0xa41368620000000000000000000000000000000000000000000000000000000000000020000000000000000000000000000000000000000000000000000000000000000c48656c6c6f20576f726c64210000000000000000000000000000000000000000'

There is also a way to directly access the variables in the storage of a contract. This also required decoding which is slightly different:
> For short byte arrays, they store their data in the same slot where the length is also stored. In particular: if the data is at most 31 bytes long, it is stored in the higher-order bytes (left aligned) and the lowest-order byte stores length * 2

* https://solidity.readthedocs.io/en/latest/miscellaneous.html?highlight=storage#bytes-and-string

The first element in storage is the bool variable (we have seen this already):

In [432]:
w3.eth.getStorageAt(caddress,0)

HexBytes('0x01')

The second element in storage is the string:

In [433]:
w3.eth.getStorageAt(caddress,1)

HexBytes('0x48656c6c6f20576f726c64210000000000000000000000000000000000000018')

In [434]:
hex(string_length * 2) # lowest-order byte stores the length * 2

'0x18'

### Function signature (aka. method ID)
The four byte *function signature* or *method id* has to be specified at the beginning of the input data. It selects the function that is invoked with the respective arguments.

In [435]:
# 4 byte function signature or method_id
len("a4136862")//2

4

In [436]:
from sha3 import keccak_256
sha3_hash = keccak_256(b"setGreeting(string)").hexdigest()
method_id = sha3_hash[:8]
method_id

'a4136862'

In [437]:
instance.functions.greetbit().call()

0

In [438]:
w3.eth.getStorageAt(caddress,2)

HexBytes('0x00')

In [439]:
tx_hash = instance.functions.setGreetbit(1).transact(
    {"from":w3.eth.accounts[0]})
tx_hash

HexBytes('0xda5fa9f2346c7fcf7509cde46fc6416d999ff744e31d40537b1aa273cd1d9a8d')

In [440]:
instance.functions.greetbit().call()

1

In [441]:
w3.eth.getStorageAt(caddress,2)

HexBytes('0x01')

In [442]:
tx_info = w3.eth.getTransaction(tx_hash)
tx_info['input']

'0x4f11dbea0000000000000000000000000000000000000000000000000000000000000001'

In [443]:
from sha3 import keccak_256
sha3_hash = keccak_256(b"setGreetbit(uint256)").hexdigest()
method_id = sha3_hash[:8]
method_id

'4f11dbea'

In [444]:
len("0000000000000000000000000000000000000000000000000000000000000001")/2 * 8

256.0

You can also manually send a transaction, this should work with `geth`

In [445]:
# This should be possible with geth but is not with web3py:
cdata = b"0x4f11dbea0000000000000000000000000000000000000000000000000000000000000000"
tx_hash = w3.eth.sendTransaction({"from":w3.eth.accounts[0],"to":caddress,"data":cdata,"value":0,"gas":100_000})
tx_hash

HexBytes('0x4a7b067906b935773378ce1afa3d2d89d18a0e81d0a9a63d107e46e42357b6cc')

In [446]:
instance.functions.greetbit().call()

0

You can also use:
* the `getData()` method in the `geth` REPL console to construct the `input` data hex string. 
* the `encodeABI()` method in `web3`

In [447]:
payload= instance.encodeABI(fn_name='setGreetbit', args=[1])
payload

'0x4f11dbea0000000000000000000000000000000000000000000000000000000000000001'

### Debug Transaction

To debug a transaction the following functions can be used: 

In [448]:
# debug i.e. trace a transaction and its operations 
w3.provider.make_request("debug_traceTransaction", [tx_hash.hex(), {}])

{'id': 117,
 'jsonrpc': '2.0',
 'result': {'gas': 28156,
  'returnValue': '',
  'structLogs': [{'depth': 0,
    'error': '',
    'gas': 77816,
    'gasCost': 3,
    'memory': [],
    'op': 'PUSH1',
    'pc': 0,
    'stack': [],
    'storage': {}},
   {'depth': 0,
    'error': '',
    'gas': 77813,
    'gasCost': 3,
    'memory': [],
    'op': 'PUSH1',
    'pc': 2,
    'stack': ['0000000000000000000000000000000000000000000000000000000000000080'],
    'storage': {}},
   {'depth': 0,
    'error': '',
    'gas': 77810,
    'gasCost': 12,
    'memory': [],
    'op': 'MSTORE',
    'pc': 4,
    'stack': ['0000000000000000000000000000000000000000000000000000000000000080',
     '0000000000000000000000000000000000000000000000000000000000000040'],
    'storage': {}},
   {'depth': 0,
    'error': '',
    'gas': 77798,
    'gasCost': 3,
    'memory': ['0000000000000000000000000000000000000000000000000000000000000000',
     '0000000000000000000000000000000000000000000000000000000000000000',
     '00

In [449]:
# debug the old way:
#w3.providers[0].make_request("debug_traceTransaction", [tx_hash.hex(), {}])

### Fallback and receive functions

The **fallback function** is the function that is invoked when the method ID does not match any other function, and some `data` is provided as parameter (or not *receive function* is defined and the *fallback function* is payable). 
For a contract to be able to accept ether this way, the fallback function has to be defined as `payable`. 
* https://docs.soliditylang.org/en/v0.8.5/contracts.html?highlight=fallback#fallback-function

> A payable fallback function is also executed for plain Ether transfers, if no receive Ether function is present.  It is recommended to always define a receive Ether function as well, if you define a payable fallback function to distinguish Ether transfers from interface confusions.

In [450]:
instance.functions.greetbit().call()

0

In [451]:
instance.functions.getGreetbool().call()

True

In [452]:
getBalance(instance.address)

0

In [453]:
cdata = b"0xffffffff" # some random data, will be ignored anyway
tx_hash = w3.eth.sendTransaction({"from":w3.eth.accounts[0],"to":caddress,"data":cdata,"value":1*10**18})
tx_hash

HexBytes('0x232e925d4b9751ea79f73e5b91b775bc31c844348a17eafcc8eebd6943ff870a')

In [454]:
instance.functions.greetbit().call()

1

In [455]:
getBalance(instance.address)

Decimal('1')

In [456]:
instance.functions.getGreetbool().call()

True

The **receive function** is the function that is invoked when the method ID does not match any other function, and **no** `data` is provided as parameter, but value transferred. 
The receive function has to be defined as `payable`. The receive function is not necessary!
* https://docs.soliditylang.org/en/v0.8.5/contracts.html?highlight=fallback#receive-ether-function

> Contracts that receive Ether directly (without a function call, i.e. using send or transfer) but do not define a receive Ether function or a payable fallback function throw an exception, sending back the Ether (this was different before Solidity v0.4.0). So if you want your contract to receive Ether, you have to implement a receive Ether function (using payable fallback functions for receiving Ether is not recommended, since it would not fail on interface confusions).

* **Warning 1:**
> In the worst case, the receive function can only rely on 2300 gas being available (for example when send or transfer is used), leaving little room to perform other operations except basic logging. The following operations will consume more gas than the 2300 gas stipend:
    + Writing to storage
    + Creating a contract
    + Calling an external function which consumes a large amount of gas
    + Sending Ether

* **Warning 2:** 
> A contract without a receive Ether function can receive Ether as a recipient of a coinbase transaction (aka miner block reward) or as a destination of a selfdestruct.
A contract cannot react to such Ether transfers and thus also cannot reject them. This is a design choice of the EVM and Solidity cannot work around it.
It also means that address(this).balance can be higher than the sum of some manual accounting implemented in a contract (i.e. having a counter updated in the receive Ether function).

In [457]:
instance.functions.greetbit().call()

1

In [458]:
instance.functions.getGreetbool().call()

True

In [459]:
tx_hash = w3.eth.sendTransaction({"from":w3.eth.accounts[0],"to":caddress,"value":1*10**18})
tx_hash

HexBytes('0x12adb2f388f752e74e682859b4a42271dec5a9135a5ebf5a1f2484b7fed400f2')

In [460]:
tx = w3.eth.getTransaction(tx_hash)
tx["gas"]

128879

In [461]:
tx_receipt = w3.eth.getTransactionReceipt(tx_hash)
tx_receipt["gasUsed"]

13879

In [462]:
instance.functions.getGreetbool().call()

False

In [463]:
instance.functions.greetbit().call()

1

#### Sending and recieveing ether to contracts
https://solidity-by-example.org/sending-ether/


# Contract deployment and interaction (the easy way) 

Now the easy way using a small utility lib

In [464]:
cat {EXAMPLE_PATH}/Greeter/Greeter.sol

// SPDX-License-Identifier: MIT
pragma solidity ^0.8.4;

contract Greeter {
    bool private greetbool;
    string public greeting;
    uint256 public greetbit;

    constructor() {
        greeting = 'Hello';
    }

    function setGreeting(string memory _greeting) public {
        greeting = _greeting;
    }

    function getGreeting() view public returns (string memory) {
        return greeting;
    }

    function setGreetbool(bool _bool) public {
        greetbool = _bool;
    }

    function getGreetbool() view public returns (bool) {
        return greetbool; 
    }

    function setGreetbit(uint256 _bit) public {
        greetbit = _bit;
    }

    fallback() external payable {
        greetbit = greetbit ^ 1;
    }

    receive() external payable {
        greetbool = !greetbool;
    }
}


In [465]:
import util as util

In [466]:
util.connect(host="172.18.0.2",port=8545,poa=False)
# ConciseContract deployment and instanceiation:
instance = util.compile_and_deploy_contract(EXAMPLE_PATH + "/Greeter/Greeter.sol")

In [467]:
instance.address

'0x22d5C8BdD4346b390014a07109a8F830094d4abf'

In [468]:
instance.getGreeting()

'Hello'

In [469]:
# With the patched concise contract Factory the function
# waits till a block is mined and directly returns the tx receipt
tx_receipt = instance.setGreeting("Nihao", transact={"from": w3.eth.accounts[0]})
tx_receipt

AttributeDict({'transactionHash': HexBytes('0x6698d16469acc2946fade1e5ea135bd390c78deb312420d4761dcf76e0f46c3f'),
 'transactionIndex': 0,
 'blockHash': HexBytes('0xa8bcdbc0c8d06d6badb86eb2ef0ddf89bccb3893e5c4065267b4eec830a406a8'),
 'blockNumber': 45,
 'from': '0x90F8bf6A479f320ead074411a4B0e7944Ea8c9C1',
 'to': '0x22d5C8BdD4346b390014a07109a8F830094d4abf',
 'gasUsed': 29586,
 'cumulativeGasUsed': 29586,
 'contractAddress': None,
 'logs': [],
 'status': 1,
 'logsBloom': HexBytes('0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000'

In [470]:
instance.getGreeting()

'Nihao'

In [471]:
# Verbose deployment and instanciation:
instance = util.compile_and_deploy_contract(EXAMPLE_PATH + "/Greeter/Greeter.sol",concise=False)

In [472]:
instance.address

'0xB9bdBAEc07751F6d54d19A6B9995708873F3DE18'

In [473]:
instance.functions.getGreeting().call()

'Hello'

In [474]:
tx_hash = instance.functions.setGreeting("Nihao").transact({"from":w3.eth.accounts[0],
                                                            "value":0,
                                                            "gas":1_000_000})
tx_hash

HexBytes('0xff202a0f338c076d7ef2a2f4bc8188668afd8cfa1705c079f0813e85addbb762')

In [475]:
instance.functions.getGreeting().call()

'Nihao'

In [476]:
# Verbose instanciation of already deployed contract (ABI generated from soruce): 
instance = util.get_contract_instance(instance.address,None,path=EXAMPLE_PATH + "/Greeter/Greeter.sol",concise=False)

In [477]:
instance.functions.getGreeting().call()

'Nihao'

# Events

Events are a way to notify clients listening to new blocks about smart contract operations that have been performed.
* https://solidity.readthedocs.io/en/develop/contracts.html#events
* https://web3py.readthedocs.io/en/stable/filters.html?highlight=events
* https://web3py.readthedocs.io/en/stable/contracts.html#events

In [478]:
!cat {EXAMPLE_PATH}/Eventer/Eventer.sol

// SPDX-License-Identifier: MIT
pragma solidity ^0.8.4;

contract Eventer {

    event create(uint256 v);
    event funcall(address caller,uint256 value);
    event funcall2(address indexed caller,uint256 indexed value);
    event fallcall(address caller);

    constructor() {
        emit create(1);
    }

    function func1(uint256 v) public {
        emit funcall(msg.sender,v);
    }

    function func2(uint256 v) public {
        emit funcall2(msg.sender,v);
    }

    receive() external payable{
        emit fallcall(msg.sender);
    }

}


In [479]:
!solc --bin {EXAMPLE_PATH}/Eventer/Eventer.sol


======= /smartenv/examples//Eventer/Eventer.sol:Eventer =======
Binary:
608060405234801561001057600080fd5b507f780900dcfb922770b66b73546245c9d725e14dd206326f4f8f5a706976c2b61d6001604051610041919061005d565b60405180910390a1610094565b61005781610082565b82525050565b6000602082019050610072600083018461004e565b92915050565b6000819050919050565b600061008d82610078565b9050919050565b61026e806100a36000396000f3fe60806040526004361061002d5760003560e01c806316d93ade14610070578063254e43db146100995761006b565b3661006b577f558048e62c62b6dedcadcb7b740131fd129cfa4019d5a5158a5befc2976d0f1d3360405161006191906101a1565b60405180910390a1005b600080fd5b34801561007c57600080fd5b506100976004803603810190610092919061015a565b6100c2565b005b3480156100a557600080fd5b506100c060048036038101906100bb919061015a565b610109565b005b803373ffffffffffffffffffffffffffffffffffffffff167f1a7f26d3c848a605ab293135411d69990f60fa592874ca544b2f290579fbe1b860405160405180910390a350565b7fcdb58a22429d89e572d17333db36ab8054687b985157808a3aefd24cdbe17520338

In [480]:
# create a contract instance 
einstance = util.compile_and_deploy_contract(EXAMPLE_PATH + "/Eventer/Eventer.sol",concise=False)

In [481]:
einstance.address

'0xFcCeD5E997E7fb1D0594518D3eD57245bB8ed17E'

Now we create a listener to listen for the latest *create* events and return all of them.

In [482]:
event_filter = einstance.events.create.createFilter(fromBlock='latest')

In [483]:
event_filter.get_all_entries()

[AttributeDict({'args': AttributeDict({'v': 1}),
  'event': 'create',
  'logIndex': 0,
  'transactionIndex': 0,
  'transactionHash': HexBytes('0xd92bb59cf9520eb02ef96929a6f1cc95fb13f3173e754432a2bddb0211601ef1'),
  'address': '0xFcCeD5E997E7fb1D0594518D3eD57245bB8ed17E',
  'blockHash': HexBytes('0x8084d4671a8b81c17ff21bd21b29ed87f3262889c2f4c4be28a59ef5ac3c378e'),
  'blockNumber': 48})]

Now we fire a new event by creating a new transaction and keep the `tx_receipt` which we cen process later on. 

In [484]:
tx_hash = einstance.functions.func1(3).transact({"from":w3.eth.coinbase})
print(tx_hash.hex())
tx_receipt = w3.eth.waitForTransactionReceipt(tx_hash)
tx_receipt

0xfa743e0e742c777d38102a04b52dcf687b06bc63a79f6f357ce5efef2b1dfbef


AttributeDict({'transactionHash': HexBytes('0xfa743e0e742c777d38102a04b52dcf687b06bc63a79f6f357ce5efef2b1dfbef'),
 'transactionIndex': 0,
 'blockHash': HexBytes('0x22bf3129a02198ef3f66dbc0bb834ce94bda4bba7838295d136c9d7a94df8658'),
 'blockNumber': 49,
 'from': '0x90F8bf6A479f320ead074411a4B0e7944Ea8c9C1',
 'to': '0xFcCeD5E997E7fb1D0594518D3eD57245bB8ed17E',
 'gasUsed': 23208,
 'cumulativeGasUsed': 23208,
 'contractAddress': None,
 'logs': [AttributeDict({'logIndex': 0,
   'transactionIndex': 0,
   'transactionHash': HexBytes('0xfa743e0e742c777d38102a04b52dcf687b06bc63a79f6f357ce5efef2b1dfbef'),
   'blockHash': HexBytes('0x22bf3129a02198ef3f66dbc0bb834ce94bda4bba7838295d136c9d7a94df8658'),
   'blockNumber': 49,
   'address': '0xFcCeD5E997E7fb1D0594518D3eD57245bB8ed17E',
   'data': '0x00000000000000000000000090f8bf6a479f320ead074411a4b0e7944ea8c9c10000000000000000000000000000000000000000000000000000000000000003',
   'topics': [HexBytes('0xcdb58a22429d89e572d17333db36ab8054687b985157808a3

In [485]:
# concise way, not further supported with events:
#tx_receipt = einstance.func1(3,transact={"from":w3.eth.coinbase})
#tx_receipt

The `tx_receipt` contains the respective event.

In [486]:
einstance.events.funcall().processReceipt(tx_receipt)

(AttributeDict({'args': AttributeDict({'caller': '0x90F8bf6A479f320ead074411a4B0e7944Ea8c9C1',
   'value': 3}),
  'event': 'funcall',
  'logIndex': 0,
  'transactionIndex': 0,
  'transactionHash': HexBytes('0xfa743e0e742c777d38102a04b52dcf687b06bc63a79f6f357ce5efef2b1dfbef'),
  'address': '0xFcCeD5E997E7fb1D0594518D3eD57245bB8ed17E',
  'blockHash': HexBytes('0x22bf3129a02198ef3f66dbc0bb834ce94bda4bba7838295d136c9d7a94df8658'),
  'blockNumber': 49}),)

The `tx_receipt` does **not** contain the respective event.

In [487]:
einstance.events.funcall2().processReceipt(tx_receipt)

/smartenv/venv/lib/python3.8/site-packages/web3/contract.py:1166: UserWarning: The log with transaction hash: HexBytes('0xfa743e0e742c777d38102a04b52dcf687b06bc63a79f6f357ce5efef2b1dfbef') and logIndex: 0 encountered the following error during processing: MismatchedABI(The event signature did not match the provided ABI). It has been discarded.
  warnings.warn(


()

Now lets fire the `funcall` event again collect all events until now

In [488]:
tx_hash = einstance.functions.func1(4).transact({"from":w3.eth.coinbase})
print(tx_hash.hex())
tx_receipt = w3.eth.waitForTransactionReceipt(tx_hash)
tx_receipt

0x3fd4c6b36a0a71a49e8b5fa9e4b86b2119bd19abc4dd31acbe851a9383ec2d93


AttributeDict({'transactionHash': HexBytes('0x3fd4c6b36a0a71a49e8b5fa9e4b86b2119bd19abc4dd31acbe851a9383ec2d93'),
 'transactionIndex': 0,
 'blockHash': HexBytes('0x81f624011d356853b15a19eadcfaf39e954c7fe8d84f715b19d431b933f2efd0'),
 'blockNumber': 50,
 'from': '0x90F8bf6A479f320ead074411a4B0e7944Ea8c9C1',
 'to': '0xFcCeD5E997E7fb1D0594518D3eD57245bB8ed17E',
 'gasUsed': 23208,
 'cumulativeGasUsed': 23208,
 'contractAddress': None,
 'logs': [AttributeDict({'logIndex': 0,
   'transactionIndex': 0,
   'transactionHash': HexBytes('0x3fd4c6b36a0a71a49e8b5fa9e4b86b2119bd19abc4dd31acbe851a9383ec2d93'),
   'blockHash': HexBytes('0x81f624011d356853b15a19eadcfaf39e954c7fe8d84f715b19d431b933f2efd0'),
   'blockNumber': 50,
   'address': '0xFcCeD5E997E7fb1D0594518D3eD57245bB8ed17E',
   'data': '0x00000000000000000000000090f8bf6a479f320ead074411a4b0e7944ea8c9c10000000000000000000000000000000000000000000000000000000000000004',
   'topics': [HexBytes('0xcdb58a22429d89e572d17333db36ab8054687b985157808a3

In [489]:
event_filter = einstance.events.funcall.createFilter(fromBlock=0)

In [490]:
event_filter.get_all_entries()

[AttributeDict({'args': AttributeDict({'caller': '0x90F8bf6A479f320ead074411a4B0e7944Ea8c9C1',
   'value': 3}),
  'event': 'funcall',
  'logIndex': 0,
  'transactionIndex': 0,
  'transactionHash': HexBytes('0xfa743e0e742c777d38102a04b52dcf687b06bc63a79f6f357ce5efef2b1dfbef'),
  'address': '0xFcCeD5E997E7fb1D0594518D3eD57245bB8ed17E',
  'blockHash': HexBytes('0x22bf3129a02198ef3f66dbc0bb834ce94bda4bba7838295d136c9d7a94df8658'),
  'blockNumber': 49}),
 AttributeDict({'args': AttributeDict({'caller': '0x90F8bf6A479f320ead074411a4B0e7944Ea8c9C1',
   'value': 4}),
  'event': 'funcall',
  'logIndex': 0,
  'transactionIndex': 0,
  'transactionHash': HexBytes('0x3fd4c6b36a0a71a49e8b5fa9e4b86b2119bd19abc4dd31acbe851a9383ec2d93'),
  'address': '0xFcCeD5E997E7fb1D0594518D3eD57245bB8ed17E',
  'blockHash': HexBytes('0x81f624011d356853b15a19eadcfaf39e954c7fe8d84f715b19d431b933f2efd0'),
  'blockNumber': 50})]

In [491]:
tx_hash = w3.eth.sendTransaction({"from":w3.eth.coinbase,
                                  "to":einstance.address,
                                  "value":1,
                                  "gas":1_000_000})
tx_receipt = w3.eth.waitForTransactionReceipt(tx_hash)
tx_receipt

AttributeDict({'transactionHash': HexBytes('0x56b3cf68202dff442384d25d64a0d6a51765813c304159a020793316157ca1bc'),
 'transactionIndex': 0,
 'blockHash': HexBytes('0x654be2a3eb3b9531fdd5ec66d919736b5e7c3640af63367900574d6a2c65d6fa'),
 'blockNumber': 51,
 'from': '0x90F8bf6A479f320ead074411a4B0e7944Ea8c9C1',
 'to': '0xFcCeD5E997E7fb1D0594518D3eD57245bB8ed17E',
 'gasUsed': 22294,
 'cumulativeGasUsed': 22294,
 'contractAddress': None,
 'logs': [AttributeDict({'logIndex': 0,
   'transactionIndex': 0,
   'transactionHash': HexBytes('0x56b3cf68202dff442384d25d64a0d6a51765813c304159a020793316157ca1bc'),
   'blockHash': HexBytes('0x654be2a3eb3b9531fdd5ec66d919736b5e7c3640af63367900574d6a2c65d6fa'),
   'blockNumber': 51,
   'address': '0xFcCeD5E997E7fb1D0594518D3eD57245bB8ed17E',
   'data': '0x00000000000000000000000090f8bf6a479f320ead074411a4b0e7944ea8c9c1',
   'topics': [HexBytes('0x558048e62c62b6dedcadcb7b740131fd129cfa4019d5a5158a5befc2976d0f1d')],
   'type': 'mined',
   'removed': False})],


In [492]:
# get the fallback function event
einstance.events.fallcall().processReceipt(tx_receipt)

(AttributeDict({'args': AttributeDict({'caller': '0x90F8bf6A479f320ead074411a4B0e7944Ea8c9C1'}),
  'event': 'fallcall',
  'logIndex': 0,
  'transactionIndex': 0,
  'transactionHash': HexBytes('0x56b3cf68202dff442384d25d64a0d6a51765813c304159a020793316157ca1bc'),
  'address': '0xFcCeD5E997E7fb1D0594518D3eD57245bB8ed17E',
  'blockHash': HexBytes('0x654be2a3eb3b9531fdd5ec66d919736b5e7c3640af63367900574d6a2c65d6fa'),
  'blockNumber': 51}),)

# Visibility

Generally all data stored in the blockchain is visible!
Function invocations can be restricted but be careful!

* https://solidity.readthedocs.io/en/v0.4.25/contracts.html#visibility-and-getters

In [493]:
!cat {EXAMPLE_PATH}/Visibility/Visibility.sol

// SPDX-License-Identifier: MIT
pragma solidity ^0.8.4;

contract Visibility {
    uint256 constant public cpub_int = 0x10; //set at compile time
    uint256 public pub_int;   //getter is created automatically 
    uint256 internal int_int; //no getter but available in derived contracts 
    uint256 private priv_int;

    event echange(uint256 indexed v);
    event ichange(uint256 indexed v);
    event pchange(uint256 indexed v);

    constructor() payable {
        pub_int = 0x20;
        int_int = 0x30;
        priv_int = 0x40;
    }
    
    // Function _should_ only be called from external accounts/contracts
    // Although can also be called form this contract with this.
    function ext_change() external returns (uint256) {
        priv_int += 1;
        emit echange(priv_int);
        return priv_int;
    }
    
    // Internal functions cannot be called directly from external accounts/contracts
    // Only indirectly 
    function int_change() internal returns (uint256) {
     

In [494]:
!solc --bin {EXAMPLE_PATH}/Visibility/Visibility.sol


======= /smartenv/examples//Visibility/Visibility.sol:Visibility =======
Binary:
608060405260206000819055506030600181905550604060028190555061047a8061002b6000396000f3fe608060405234801561001057600080fd5b50600436106100625760003560e01c80630f66c3d2146100675780634c370d2514610097578063529f246f146100b5578063ebd3b3dd146100d3578063f455a815146100f1578063fcce02071461010f575b600080fd5b610081600480360381019061007c9190610322565b61013f565b60405161008e9190610383565b60405180910390f35b61009f610156565b6040516100ac9190610383565b60405180910390f35b6100bd6101a9565b6040516100ca9190610383565b60405180910390f35b6100db610285565b6040516100e89190610383565b60405180910390f35b6100f961028b565b6040516101069190610383565b60405180910390f35b61012960048036038101906101249190610322565b610290565b6040516101369190610383565b60405180910390f35b60006002548261014f919061039e565b9050919050565b600060016002600082825461016b919061039e565b925050819055506002547f437ed1f411e2ba750c55a12da4f261a1b9f2db8c9046de9ab6119538e1def345604051604051809103

In [495]:
vinstance = util.compile_and_deploy_contract(EXAMPLE_PATH + "/Visibility/Visibility.sol",concise=False)

In [496]:
hex(vinstance.functions.pub_int().call())

'0x20'

In [497]:
hex(vinstance.functions.pub_change().call())

'0x21'

In [498]:
hex(vinstance.functions.view_priv_int(1).call())

'0x41'

In [499]:
tx_hash = vinstance.functions.pub_change().transact({"from":w3.eth.coinbase,})
tx_hash
tx_receipt = w3.eth.waitForTransactionReceipt(tx_hash)
tx_receipt

AttributeDict({'transactionHash': HexBytes('0xd44954cd99a6f28ca9c6c2e47334474f2f6ce824efa74adbdedbad0ef088fcaa'),
 'transactionIndex': 0,
 'blockHash': HexBytes('0xd37ca8fedabfb3dcf99414aedd771c6f75b324bf357637ebb8a9ad0bc46cefee'),
 'blockNumber': 53,
 'from': '0x90F8bf6A479f320ead074411a4B0e7944Ea8c9C1',
 'to': '0x4cFB3F70BF6a80397C2e634e5bDd85BC0bb189EE',
 'gasUsed': 49957,
 'cumulativeGasUsed': 49957,
 'contractAddress': None,
 'logs': [AttributeDict({'logIndex': 0,
   'transactionIndex': 0,
   'transactionHash': HexBytes('0xd44954cd99a6f28ca9c6c2e47334474f2f6ce824efa74adbdedbad0ef088fcaa'),
   'blockHash': HexBytes('0xd37ca8fedabfb3dcf99414aedd771c6f75b324bf357637ebb8a9ad0bc46cefee'),
   'blockNumber': 53,
   'address': '0x4cFB3F70BF6a80397C2e634e5bDd85BC0bb189EE',
   'data': '0x',
   'topics': [HexBytes('0x437ed1f411e2ba750c55a12da4f261a1b9f2db8c9046de9ab6119538e1def345'),
    HexBytes('0x0000000000000000000000000000000000000000000000000000000000000041')],
   'type': 'mined',
   '

In [500]:
vinstance.events.pchange().processReceipt(tx_receipt)

/smartenv/venv/lib/python3.8/site-packages/web3/contract.py:1166: UserWarning: The log with transaction hash: HexBytes('0xd44954cd99a6f28ca9c6c2e47334474f2f6ce824efa74adbdedbad0ef088fcaa') and logIndex: 0 encountered the following error during processing: MismatchedABI(The event signature did not match the provided ABI). It has been discarded.
  warnings.warn(
/smartenv/venv/lib/python3.8/site-packages/web3/contract.py:1166: UserWarning: The log with transaction hash: HexBytes('0xd44954cd99a6f28ca9c6c2e47334474f2f6ce824efa74adbdedbad0ef088fcaa') and logIndex: 1 encountered the following error during processing: MismatchedABI(The event signature did not match the provided ABI). It has been discarded.
  warnings.warn(


(AttributeDict({'args': AttributeDict({'v': 33}),
  'event': 'pchange',
  'logIndex': 2,
  'transactionIndex': 0,
  'transactionHash': HexBytes('0xd44954cd99a6f28ca9c6c2e47334474f2f6ce824efa74adbdedbad0ef088fcaa'),
  'address': '0x4cFB3F70BF6a80397C2e634e5bDd85BC0bb189EE',
  'blockHash': HexBytes('0xd37ca8fedabfb3dcf99414aedd771c6f75b324bf357637ebb8a9ad0bc46cefee'),
  'blockNumber': 53}),)

In [501]:
vinstance.events.echange().processReceipt(tx_receipt)

/smartenv/venv/lib/python3.8/site-packages/web3/contract.py:1166: UserWarning: The log with transaction hash: HexBytes('0xd44954cd99a6f28ca9c6c2e47334474f2f6ce824efa74adbdedbad0ef088fcaa') and logIndex: 2 encountered the following error during processing: MismatchedABI(The event signature did not match the provided ABI). It has been discarded.
  warnings.warn(


(AttributeDict({'args': AttributeDict({'v': 65}),
  'event': 'echange',
  'logIndex': 0,
  'transactionIndex': 0,
  'transactionHash': HexBytes('0xd44954cd99a6f28ca9c6c2e47334474f2f6ce824efa74adbdedbad0ef088fcaa'),
  'address': '0x4cFB3F70BF6a80397C2e634e5bDd85BC0bb189EE',
  'blockHash': HexBytes('0xd37ca8fedabfb3dcf99414aedd771c6f75b324bf357637ebb8a9ad0bc46cefee'),
  'blockNumber': 53}),)

In [502]:
vinstance.events.ichange().processReceipt(tx_receipt)

(AttributeDict({'args': AttributeDict({'v': 49}),
  'event': 'ichange',
  'logIndex': 1,
  'transactionIndex': 0,
  'transactionHash': HexBytes('0xd44954cd99a6f28ca9c6c2e47334474f2f6ce824efa74adbdedbad0ef088fcaa'),
  'address': '0x4cFB3F70BF6a80397C2e634e5bDd85BC0bb189EE',
  'blockHash': HexBytes('0xd37ca8fedabfb3dcf99414aedd771c6f75b324bf357637ebb8a9ad0bc46cefee'),
  'blockNumber': 53}),)